In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-11 23:53:22,0.43945,0.0,0.0,0.0,2022-06-11,23:53:22,23
1,2022-06-11 23:54:22,0.48828,0.0,0.0,0.0,2022-06-11,23:54:22,23
2,2022-06-11 23:55:22,0.46387,0.0,0.0,0.0,2022-06-11,23:55:22,23
3,2022-06-11 23:56:22,0.46387,0.0,0.0,0.0,2022-06-11,23:56:22,23
4,2022-06-11 23:57:22,0.46387,0.0,0.0,0.0,2022-06-11,23:57:22,23


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
16130,2022-06-23 16:19:13,2022-06-23,16:19:13,16,21.06934,0.00000,0.000000,0.000000
16131,2022-06-23 16:20:13,2022-06-23,16:20:13,16,15.82031,3.48727,55.169692,0.055170
16132,2022-06-23 16:21:13,2022-06-23,16:21:13,16,21.06934,0.00000,0.000000,0.000000
16133,2022-06-23 16:22:13,2022-06-23,16:22:13,16,15.40527,3.90245,60.118296,0.060118
16134,2022-06-23 16:23:13,2022-06-23,16:23:13,16,21.09375,3.36158,70.908328,0.070908


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-11T00:00:00.000000000', '2022-06-12T00:00:00.000000000',
       '2022-06-13T00:00:00.000000000', '2022-06-14T00:00:00.000000000',
       '2022-06-15T00:00:00.000000000', '2022-06-16T00:00:00.000000000',
       '2022-06-17T00:00:00.000000000', '2022-06-18T00:00:00.000000000',
       '2022-06-19T00:00:00.000000000', '2022-06-20T00:00:00.000000000',
       '2022-06-21T00:00:00.000000000', '2022-06-22T00:00:00.000000000',
       '2022-06-23T00:00:00.000000000'], dtype='datetime64[ns]')

In [27]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

In [7]:
filter_daily_values = df[(df['Date'] >= '2022-06-18') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
8169,2022-06-18,0,0.0
8170,2022-06-18,0,0.0
8171,2022-06-18,0,0.0
8172,2022-06-18,0,0.0
8173,2022-06-18,0,0.0
...,...,...,...
15187,2022-06-22,22,0.0
15188,2022-06-22,22,0.0
15189,2022-06-22,22,0.0
15190,2022-06-22,22,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values.head()

,Date,Hour,Power (KW)
0,2022-06-18,0,0.0
1,2022-06-18,1,0.0
2,2022-06-18,2,0.0
3,2022-06-18,3,0.0
4,2022-06-18,4,0.0


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail()

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
139,42.2,21.1,53,25
140,16.5,20.2,60,70
141,5.2,19.5,69,70
142,0.0,18.1,78,68
143,0.0,16.8,88,70


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(14)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
130,NaN,82.9,21.9,53,11
131,NaN,80.4,23.5,45,25
132,NaN,84.1,25.0,40,25
133,NaN,85.4,26.0,37,25
134,NaN,84.3,26.3,35,25
135,NaN,80.7,26.1,33,25
136,NaN,74.7,24.9,37,25
137,NaN,66.3,24.0,39,25
138,NaN,55.5,22.7,44,25
139,NaN,42.2,21.1,53,25


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
reg = linear_model.LinearRegression(fit_intercept = False)
reg.fit(independent_columns, dependent_column)

LinearRegression(fit_intercept=False)

In [16]:
r_sq = reg.score(independent_columns, dependent_column)
r_sq

0.355960589189784

In [17]:
reg.intercept_

0.0

In [18]:
reg.coef_

array([ 0.01738685,  0.04213924, -0.0017386 , -0.0081179 ])

In [19]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [20]:
return_array = list(reg.predict(forcasted_data))

In [21]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [22]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [23]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [24]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [25]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-23,0,0.000000
1,2022-06-23,1,0.000000
2,2022-06-23,2,0.000000
3,2022-06-23,3,0.000000
4,2022-06-23,4,0.000000
5,2022-06-23,5,0.541865
6,2022-06-23,6,0.980895
7,2022-06-23,7,1.408004
8,2022-06-23,8,1.632652
9,2022-06-23,9,1.940484


### MSE = mean_squared_error(actual value, predicted value)

In [35]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(17))
mean_sq_error

0.9991952631265946

In [36]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.9995975505805297

### R squared

In [37]:
r_square = metrics.r2_score(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(17))
r_square

-0.5521309714109863

In [ ]:
# dependent_variable = 'Power (KW)'

In [ ]:
# independent_variables = df1.columns.tolist()
# independent_variables.remove(dependent_variable)

In [ ]:
# independent_variables

In [ ]:
# X = df1[independent_variables].values

In [ ]:
# y = df1[dependent_variable].values

In [ ]:
# X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# X_train

In [ ]:
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

In [ ]:
# y_pred = regressor.predict(X_test)
# y_pred

In [ ]:
# math.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# r2_score(y_test, y_pred)